In [12]:
import json
import re
import pandas as pd

def logs_to_dataframe(log_data):
    # Parse log entries
    entries = []
    for line in log_data.strip().split('\n'):
        if line.startswith('{'):
            try:
                entry = json.loads(line)
                entries.append(entry)
            except:
                pass
    
    # Extract data for orders DataFrame
    orders_data = []
    errors_data = []
    
    for entry in entries:
        timestamp = entry.get('timestamp')
        lambda_log = entry.get('lambdaLog', '')
        sandbox_log = entry.get('sandboxLog', '')
        
        # Extract orders
        for match in re.findall(r'SELL (\d+)x (\d+)', lambda_log):
            quantity, price = match
            orders_data.append({
                'timestamp': timestamp,
                'quantity': int(quantity),
                'price': int(price),
                'type': 'SELL'
            })
        
        # Extract buy/sell order depths
        buy_depth_match = re.search(r'Buy Order depth : (\d+)', lambda_log)
        sell_depth_match = re.search(r'Sell order depth : (\d+)', lambda_log)
        
        if buy_depth_match and sell_depth_match:
            buy_depth = int(buy_depth_match.group(1))
            sell_depth = int(sell_depth_match.group(1))
            
            # Add to orders_data if not already captured
            if not any(d['timestamp'] == timestamp for d in orders_data):
                orders_data.append({
                    'timestamp': timestamp,
                    'buy_depth': buy_depth,
                    'sell_depth': sell_depth
                })
            else:
                # Update existing entries
                for order in orders_data:
                    if order['timestamp'] == timestamp:
                        order['buy_depth'] = buy_depth
                        order['sell_depth'] = sell_depth
        
        # Extract errors
        for product in re.findall(r'Orders for product (\w+) exceeded', sandbox_log):
            errors_data.append({
                'timestamp': timestamp,
                'product': product,
                'error': 'exceeded limit'
            })
    
    # Create DataFrames
    orders_df = pd.DataFrame(orders_data)
    errors_df = pd.DataFrame(errors_data) if errors_data else pd.DataFrame(columns=['timestamp', 'product', 'error'])
    
    return orders_df, errors_df

In [13]:
with open('94c5bd59-6268-43e3-b99a-3b5abc114c78.log', 'r') as f:
        log_data = f.read()
    
orders_df, errors_df = logs_to_dataframe(log_data)


In [18]:
print(orders_df)


Empty DataFrame
Columns: []
Index: []
